<a href="https://colab.research.google.com/github/maliksalman55/Thesis/blob/main/New_Dataset_Generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import os
import shutil
import math

In [ ]:
src_path = '/content/drive/MyDrive/Project/kaggle.json'
dest_directory = '/root/.kaggle/'

In [ ]:
# Create the destination directory if it doesn't exist
os.makedirs(dest_directory, exist_ok=True)

In [ ]:
# Copy the Kaggle JSON file to the destination
shutil.copy(src_path, dest_directory)

'/root/.kaggle/kaggle.json'

In [ ]:
# run this for dataset download
!kaggle datasets download -d markdaniellampa/fish-dataset

 99% 1.57G/1.58G [00:19<00:00, 41.7MB/s]
100% 1.58G/1.58G [00:19<00:00, 86.4MB/s]


In [ ]:
# unzip download file
import zipfile
zip_ref = zipfile.ZipFile ('/content/fish-dataset.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
# Define the directory containing our fish images
train_dir = '/content/drive/MyDrive/Project/FishImgDataset/train'

In [ ]:
# Define the output directory for organized classes
output_dir = '/content/drive/MyDrive/Project/FishImgDataset/w8dataset'  # Creating this directory to organize images


In [ ]:
# Function to estimate weight from length and girth
def estimate_weight(length_pixels, girth_pixels):
    # Pixel-to-inch conversion factor (calibration factor)
    pixel_to_inch_conversion = 0.1  # Example: 0.1 inch per pixel

    # Convert pixel measurements to inches
    length_in_inches = length_pixels * pixel_to_inch_conversion
    girth_in_inches = girth_pixels * pixel_to_inch_conversion

    # Calculate the weight using the formula for slender fish (e.g., trout, northern pike)
    weight = (length_in_inches * girth_in_inches * girth_in_inches) / 900
    return weight

In [ ]:
# Iterate through subdirectories in the train directory
for subdir in os.listdir(train_dir):
    subdir_path = os.path.join(train_dir, subdir)

    if os.path.isdir(subdir_path):
        # Iterate through images in the subdirectory
        for filename in os.listdir(subdir_path):
            if any(filename.lower().endswith(ext) for ext in (".jpg", ".jpeg", ".png")):

                image_path = os.path.join(subdir_path, filename)

                # Load the fish image
                image = cv2.imread(image_path)

                if image is not None:
                    # Applying image processing steps (segmentation, length, girth estimation, etc.)

                    # Define coordinates for cropping (x, y, width, height)
                    x, y, w, h = 5, 10, 400, 300  # Adjust these values as needed

                    # Check if cropping region is within image boundaries
                    if x >= 0 and y >= 0 and x + w <= image.shape[1] and y + h <= image.shape[0]:
                        # Crop the fish region
                        cropped_fish = image[y:y+h, x:x+w]

                        # Continue with image segmentation
                        # Convert the cropped image to grayscale for thresholding
                        gray = cv2.cvtColor(cropped_fish, cv2.COLOR_BGR2GRAY)

                        # Apply thresholding to segment the fish from the background
                        _, binary_mask = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)

                        # Find contours in the binary mask
                        contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                        if len(contours) > 0:
                            # Find the largest contour (assuming it corresponds to the fish)
                            largest_contour = max(contours, key=cv2.contourArea)

                            # Create a mask for the fish region
                            mask = np.zeros_like(binary_mask)
                            cv2.drawContours(mask, [largest_contour], 0, 255, -1)

                            # Bitwise AND to extract the fish region from the original image
                            segmented_fish = cv2.bitwise_and(cropped_fish, cropped_fish, mask=mask)

                            # Enhance Contrast and Brightness
                            alpha = 1.5  # Contrast control
                            beta = 30    # Brightness control

                            enhanced_fish = cv2.convertScaleAbs(cropped_fish, alpha=alpha, beta=beta)

                            # Convert the segmented fish image to grayscale for edge detection
                            gray = cv2.cvtColor(segmented_fish, cv2.COLOR_BGR2GRAY)

                            # Apply Canny edge detection
                            edges = cv2.Canny(gray, 50, 150)

                            # Find contours in the edge-detected image
                            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                            if len(contours) > 0:
                                # Find the contour with the maximum length (assuming it corresponds to the fish)
                                max_contour = max(contours, key=lambda x: cv2.arcLength(x, closed=True))

                                # Measure the length of the contour
                                length = cv2.arcLength(max_contour, closed=True)

                                # Estimate girth and weight
                                #estimating girth
                                largest_contour = max(contours, key=cv2.contourArea)
                                girth_area = cv2.contourArea(largest_contour)
                                diameter = math.sqrt((4 * girth_area) / math.pi) #2*r
                                girth = math.pi * diameter

                                # Estimate weight
                                weight = estimate_weight(length, girth)

                                # Determine the folder based on the estimated weight (rounding to the nearest integer)
                                weight_folder = os.path.join(output_dir, f"{round(weight)}_pounds")
                                os.makedirs(weight_folder, exist_ok=True)

                                # Move the image to the appropriate folder
                                destination_path = os.path.join(weight_folder, filename)
                                shutil.move(image_path, destination_path)

                                print(f"Processed: {filename} | Estimated Weight: {weight} pounds")

print("Processing complete. Images organized by weight.")

Processed: 1379993778_114.79.18.246-4285884.jpeg | Estimated Weight: 1.2266403730927333 pounds
Processed: 1395692377_190.80.8.9-2959810.jpeg | Estimated Weight: 0.42755643618185823 pounds
Processed: 1369546354_182.178.75.77-567248.jpeg | Estimated Weight: 37.175463168304255 pounds
Processed: 1383035272_110.143.155.185-1813099.jpeg | Estimated Weight: 17.936484094814535 pounds
Processed: 1406026196633Ex7bofVGQQZcvfA9-4207811.jpeg | Estimated Weight: 55.064955604876495 pounds
Processed: 14060261958868RI79raSey4Sqa8d-3811978.jpeg | Estimated Weight: 6.182674799436497 pounds
Processed: 1371184733_182.178.7.254-5957736.jpeg | Estimated Weight: 19.136866001994324 pounds
Processed: 1341932090_89.244.87.39-4646055.jpeg | Estimated Weight: 36.39189571085062 pounds
Processed: 1363195128_182.178.27.53-3485615.jpeg | Estimated Weight: 2.9449725182672926 pounds
Processed: 1379743278_181.66.101.43-832254.jpeg | Estimated Weight: 1.1963831670438114 pounds
Processed: 1341932227_89.244.87.39-2340584.jp